<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/test_app_web3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -qqq web3

In [73]:
import numpy as np
import time
import requests, json

In [6]:
from web3 import Web3

# --- 1. تنظیمات اولیه با Alchemy ---

# آدرس نود اتریوم از طریق Alchemy
# توجه: کلید API خود را جایگزین 'YOUR_ALCHEMY_API_KEY' کنید
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/1d3m81eR2Kd2mrpuH6-bR"
web3 = Web3(Web3.HTTPProvider(alchemy_url))

# بررسی اتصال به شبکه
if not web3.is_connected():
    print("dont connect through alchemy")
    exit()
else:
    print("connected")


# --- 2. اطلاعات استخر ---

# آدرس قرارداد استخر مورد نظر در یونی‌سواپ V3
# برای مثال، از استخر USDC/WETH با کارمزد 0.05% استفاده شده است
pool_address = "0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082"


# --- 3. ABI قرارداد استخر ---

# ABI (Application Binary Interface) برای تعامل با توابع قرارداد هوشمند ضروری است.
# برای تابع tickSpacing()، تنها به بخش کوچکی از ABI کامل نیاز داریم.
pool_abi = [
    {
        "inputs": [],
        "name": "tickSpacing",
        "outputs": [
            {
                "internalType": "int24",
                "name": "",
                "type": "int24"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]


# --- 4. ایجاد نمونه قرارداد و فراخوانی تابع ---

# ایجاد یک نمونه از قرارداد استخر
pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)

try:
    # فراخوانی تابع 'tickSpacing' از قرارداد
    tick_spacing = pool_contract.functions.tickSpacing().call()

    # --- 5. نمایش نتیجه ---
    print("-" * 30)
    print(f"pool address: {pool_address}")
    print(f"pool Tick Spacing : {tick_spacing}")
    print("-" * 30)

except Exception as e:
    print(f"cause error : {e}")

connected
------------------------------
pool address: 0x5aE13BAAEF0620FdaE1D355495Dc51a17adb4082
pool Tick Spacing : 10
------------------------------


In [7]:
GRAPH_API_KEY='0f011cf8766c88189291aa0b95792828'
SUBGRAPH_ID = "5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

In [15]:
def find_key_path(data, target_key):
    """
    جستجو در دیکشنری تو در تو برای یک کلید مشخص و برگرداندن
    مسیر root-to-leaf به صورت دیکشنری نگهدارنده ساختار.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return {key: value}
            elif isinstance(value, (dict, list)):
                found = find_key_path(value, target_key)
                if found:
                    return {key: found}  # مسیر تا کلید هدف
    elif isinstance(data, list):
        for item in data:
            found = find_key_path(item, target_key)
            if found:
                return found
    return None


In [16]:
def find_key_path(data, target_key):
    """
    جستجو در دیکشنری تو در تو برای یک کلید مشخص و برگرداندن
    مسیر root-to-leaf به صورت دیکشنری نگهدارنده ساختار.
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return {key: value}
            elif isinstance(value, (dict, list)):
                found = find_key_path(value, target_key)
                if found:
                    return {key: found}  # مسیر تا کلید هدف
    elif isinstance(data, list):
        for item in data:
            found = find_key_path(item, target_key)
            if found:
                return found
    return None


In [24]:
import requests, json

url = f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

pool_id = "0x5ae13baaef0620fdae1d355495dc51a17adb4082" # آدرس استخر (باید lowercase باشد)

query = f"""
{{
  pool(id: "{pool_id}") {{
    id
    createdAtTimestamp        # 1. زمان یونیکس ایجاد استخر (برای محاسبه عمر)
    tick                      # 2. تیک فعال فعلی (برای محاسبه ضریب تمرکز)
    feeTier                   # 3. نرخ کارمزد استخر (مثلاً 3000)
    feesUSD                   # 4. کارمزد کل انباشته تولید شده از ابتدا (صورت کسر APR)
    totalValueLockedUSD       # 5. کل نقدینگی قفل شده (TVL) به دلار (مخرج کسر APR)
    token0Price
    token1Price
    sqrtPrice
    token0 {{
      symbol                  # 6. نماد توکن 0 (PAXG)
      decimals                # 7. اعشار توکن 0 (برای تبدیل L به دلار)
    }}
    token1 {{
      symbol                  # 8. نماد توکن 1 (USDC)
      decimals                # 9. اعشار توکن 1 (برای تبدیل L به دلار)
    }}
  }}
}}
"""

r = requests.post(url, json={"query": query})
print(json.dumps(r.json(), indent=2))

{
  "data": {
    "pool": {
      "createdAtTimestamp": "1655264999",
      "feeTier": "500",
      "feesUSD": "505709.7764222105",
      "id": "0x5ae13baaef0620fdae1d355495dc51a17adb4082",
      "sqrtPrice": "5113103145064774945969400",
      "tick": "-192976",
      "token0": {
        "decimals": "18",
        "symbol": "PAXG"
      },
      "token0Price": "0.0002400988390264598896841108648339697",
      "token1": {
        "decimals": "6",
        "symbol": "USDC"
      },
      "token1Price": "4164.951417735909355842519050046569",
      "totalValueLockedUSD": "11753265.53898257123034187226545919"
    }
  }
}


In [57]:
fee_tier=float(find_key(r.json(), "feeTier")[0])
tvl=float(find_key(r.json(), "totalValueLockedUSD")[0])
build_date=float(find_key(r.json(), "createdAtTimestamp")[0])
decimal0=float(find_key(r.json(), "decimals")[0])
decimal1=float(find_key(r.json(), "decimals")[1])
current_tick=float(find_key(r.json(), "tick")[0])
current_price=float(find_key(r.json(), "token1Price")[0])
sqrtprice=float(find_key(r.json(), "sqrtPrice")[0])

In [52]:
def calculate_price_by_tick(tick:int,decimal_0:int,decimal_1:int):
  return (10**(decimal_0-decimal_1))*(1.0001**tick)
  '''if decimal_0>decimal_1:
    return (10**(decimal_0-decimal_1))*(1.0001**tick)
  else:
    return (10**(decimal_1-decimal_0))*(1.0001**tick)'''

In [53]:
def calculate_tick_by_price(price:float,decimal_0:int,decimal_1:int):
  t=(np.log(price)-np.log((10**(decimal_0-decimal_1))))/(np.log(1.0001))
  if t>0:
    return np.ceil(t)
  else:
    return np.floor(t)
  """if decimal_0>decimal_1:
    t=(np.log(price)-np.log((10**(decimal_0-decimal_1))))/(np.log(1.0001))
    if t>0:
      return np.ceil(t)
    else:
      return np.floor(t)
  else:
    t=(np.log(price)-np.log((10**(decimal_1-decimal_0))))/(np.log(1.0001))
    if t>0:
      return np.floor(t)
    else:
      return np.ceil(t)"""


In [67]:
def calculate_price_from_sqrt_price(sqrt_price_raw: float, decimal_0: int, decimal_1: int) -> float:

    normalized_sqrt_price = sqrt_price_raw / (2 ** 96)

    raw_price_math = normalized_sqrt_price ** 2

    decimal_adjustment = decimal_0 - decimal_1

    final_price = raw_price_math * (10 ** decimal_adjustment)

    return final_price

In [62]:
calculate_price_by_tick(current_tick,decimal0,decimal1)

4164.643395510395

In [63]:
calculate_tick_by_price(current_price,decimal0,decimal1)

np.float64(-192976.0)

In [68]:
calculate_price_from_sqrt_price(sqrtprice,decimal0,decimal1)

4164.951417735911

In [78]:
def pool_base_apr(inp_pool_id:str):


  urlv = f"https://gateway.thegraph.com/api/{GRAPH_API_KEY}/subgraphs/id/{SUBGRAPH_ID}"

  pool_idv = inp_pool_id.lower()

  queryv = f"""
  {{
    pool(id: "{pool_idv}") {{
      id
      createdAtTimestamp        # 1. زمان یونیکس ایجاد استخر (برای محاسبه عمر)
      tick                      # 2. تیک فعال فعلی (برای محاسبه ضریب تمرکز)
      feeTier                   # 3. نرخ کارمزد استخر (مثلاً 3000)
      feesUSD                   # 4. کارمزد کل انباشته تولید شده از ابتدا (صورت کسر APR)
      totalValueLockedUSD       # 5. کل نقدینگی قفل شده (TVL) به دلار (مخرج کسر APR)
      token0Price
      token1Price
      sqrtPrice
      token0 {{
        symbol                  # 6. نماد توکن 0 (PAXG)
        decimals                # 7. اعشار توکن 0 (برای تبدیل L به دلار)
      }}
      token1 {{
        symbol                  # 8. نماد توکن 1 (USDC)
        decimals                # 9. اعشار توکن 1 (برای تبدیل L به دلار)
      }}
    }}
  }}
  """

  rv = requests.post(url, json={"query": query})

  #fee_tierv=float(find_key(rv.json(), "feeTier")[0])
  tvlv=float(find_key(rv.json(), "totalValueLockedUSD")[0])
  build_datev=float(find_key(rv.json(), "createdAtTimestamp")[0])
  #decimal0v=float(find_key(rv.json(), "decimals")[0])
  #decimal1v=float(find_key(rv.json(), "decimals")[1])
  #current_tickv=float(find_key(rv.json(), "tick")[0])
  #current_pricev=float(find_key(rv.json(), "token1Price")[0])
  #sqrtpricev=float(find_key(rv.json(), "sqrtPrice")[0])
  all_fee_usdv=float(find_key(rv.json(), "feesUSD")[0])
  #cpf=calculate_price_from_sqrt_price(sqrtpricev,decimal0v,decimal1v)
  now_time=time.time()
  days=(int(now_time)-build_datev)/(24*60*60)
  return (((all_fee_usdv/days)/tvl)+1)**365,((((all_fee_usdv/days)/tvl))*365)+1

In [84]:
base_apy,base_apr=pool_base_apr(pool_address)

In [89]:
import math

# نرخ پایه Uniswap (1.0001) به توان 0.5
Q_HALF = 1.0001 ** 0.5

def calculate_concentrated_apr_final(
    base_apr: float,
    current_tick: int,
    target_spacing: int,
    wide_ref_ticks: int = 500
) -> float:
    """
    محاسبه APR متمرکز نهایی با اعمال قدر مطلق بر ضریب تمرکز (Multiplier)
    برای جلوگیری از خطای ممیز شناور (Catastrophic Cancellation).
    """

    # --- 1. تعریف بازه تیک‌ها ---

    # بازه مرجع (500 تیک)
    ref_low_tick = current_tick - (wide_ref_ticks // 2)
    ref_high_tick = current_tick + (wide_ref_ticks // 2)

    # بازه هدف (Target Spacing)
    target_low_tick = current_tick
    target_high_tick = current_tick + target_spacing


    # --- 2. محاسبه Delta Sqrt Price برای بازه مرجع (Wide) ---

    # SqrtPrice = 1.0001 ** (tick / 2.0)
    sqrt_p_ref_high = 1.0001 ** (ref_high_tick / 2.0)
    sqrt_p_ref_low = 1.0001 ** (ref_low_tick / 2.0)

    # تفاضل (باید مثبت باشد، اما ممکن است به دلیل خطا منفی شود)
    delta_sqrt_price_wide = sqrt_p_ref_high - sqrt_p_ref_low


    # --- 3. محاسبه Delta Sqrt Price برای بازه هدف (Narrow) ---

    sqrt_p_target_high = 1.0001 ** (target_high_tick / 2.0)
    sqrt_p_target_low = 1.0001 ** (target_low_tick / 2.0)

    delta_sqrt_price_narrow = sqrt_p_target_high - sqrt_p_target_low


    # --- 4. محاسبه ضریب تمرکز ---
    if delta_sqrt_price_narrow == 0:
        return 0.0

    # اعمال قدر مطلق برای رفع خطای علامت منفی
    concentration_multiplier = abs(delta_sqrt_price_wide / delta_sqrt_price_narrow)

    # 5. محاسبه APR متمرکز نهایی
    apr_concentrated = base_apr * concentration_multiplier

    return apr_concentrated

In [93]:
calculate_concentrated_apr_final((base_apr-1),current_tick,10)

0.6230377415306584

In [94]:
calculate_concentrated_apr_final(0.18,current_tick,10)

8.997984594789736

In [95]:
calculate_concentrated_apr_final(0.12,current_tick,10)

5.99865639652649